# Task 1: implement your own LeNet-5 for MNIST dataset.


*   Change all architecture choices
  * number of filters
  * size of filters
  * activation (use RELU)
  * keep the pooling (size and stride) and the padding
  * use dropout (dropout rate = 0.5)
  * use L2 regularization for dense layers (except last one)




# Fiona Nicdao Assigment 3: CNN LeNet-5

In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import time

## processing the MNIST Dataset
* normalize the data
* 70% training and 30% testing sets

In [25]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# normalize the data
x_train = np.expand_dims((x_train/255.0),axis=3) # add channel for color
x_test = np.expand_dims((x_test/255.0),axis=3)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Change the data to be split into 70% training set and 30% testing set
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
train_size = 0.7
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size,
                                                    random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)



(49000, 28, 28, 1)
(49000, 10)
(21000, 28, 28, 1)
(21000, 10)


 ## CNN -LeNet-5
 * Adam optimizer with 2e-4 learning rate
 * batch size = 32
 * number of training epochs = 50
 * optain the trainging and testing accuracy

In [26]:
class LeNet(tf.keras.Model):
  def __init__(self,conv_layers, pooling_layers, fc_layers,
               input_shape, output_shape, activation, loss,
               output_activation, w_init,reg_lambda, dropout):
    super(LeNet, self).__init__()
    self.conv_layers = conv_layers
    self.pooling_layers = pooling_layers
    self.fc_layers = fc_layers
    self.input_shape = input_shape
    self.output_shape = output_shape
    self.activation = activation
    self.optimizer = tf.keras.optimizers.Adam # set and don't change
    self.loss = loss
    self.output_activation = output_activation
    self.w_init = w_init
    self.reg_lambda = reg_lambda
    self.dropout = dropout
    self.learning_rate = 2e-4 # set and don't change
    self.batch_size = 32 # set and don't change
    self.epochs = 50 # set and don't change
    self.regularizer = tf.keras.regularizers.l2(self.reg_lambda)

    self.model = self.create_model()

  def create_model(self):
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Input(shape=self.input_shape))

      for conv, pool in zip(self.conv_layers, self.pooling_layers):
        model.add(tf.keras.layers.Conv2D(filters=conv[0], kernel_size=conv[1],
                                         padding='same',
                                         activation=self.activation,
                                         kernel_regularizer=self.regularizer,
                                         kernel_initializer=self.w_init,))
        model.add(tf.keras.layers.AveragePooling2D(pool_size=pool[0], strides=pool[1]))

      model.add(tf.keras.layers.Flatten()) # create model makes

      for fc in self.fc_layers:
        model.add(tf.keras.layers.Dense( fc, activation= self.activation,
                                         kernel_regularizer=self.regularizer,
                                         kernel_initializer=self.w_init))
      # output layer
      model.add(tf.keras.layers.Dense(self.output_shape,
                                      activation=self.output_activation,
                                      kernel_regularizer=self.regularizer,
                                      kernel_initializer=self.w_init))

      model.compile(optimizer=self.optimizer(learning_rate=self.learning_rate),
                    loss=self.loss,
                    metrics=['accuracy'])

      return model

  def train(self, x_train, y_train, x_test, y_test):
    t0 = time.time()
    history = self.model.fit(x_train, y_train,
                             epochs=self.epochs,
                             batch_size=self.batch_size,
                             validation_data=(x_test, y_test))
    print("training time ", time.time()-t0)
    end_time = time.time()-t0
    return history, end_time

  def evaluate(self, x_test, y_test):
    test_loss, test_acc = self.model.evaluate(x_test, y_test)
    return test_loss, test_acc
    # return self.model.evaluate(x_test, y_test)

## Testing the Model with different Architecture choices
* change number of filters
* change size of filters

Keep the same
* keep the pooling(size and stride) and the padding
* use dropout rate = 0.5
* use L2 regularizatio for dense layers (except last one)

In [32]:
# make a dataframe to compare the accuracy and loss for all the different architecture choices
df = pd.DataFrame(columns=['cov_layers','pooling_layers', 'fc_layers', 'Loss', 'Accuracy','Time'])
print(df)

Empty DataFrame
Columns: [cov_layers, pooling_layers, fc_layers, Loss, Accuracy, Time]
Index: []


### Test: Model 1

In [33]:
# filter_size, num_filters, input_shape, out_Shape, stride, padding
# pooling and window_size, stride, activation, optim, loss, learning rate , batch size
# epochs, regularizer, reg_lambda - strength of regularization, dropout, weight initializer

conv_layers = [[6,5], [16,5]] # number of filters , 5x5  filers
pooling_layers = [[2, 2],[2,2]] # 2x2 and stride by 2 / we have two pooling layers with 2x2 window  and 2 stride
fc_layers = [120, 84] # dense layer
input_shape = (28,28,1)
output_shape = 10
activation = 'relu'
loss = 'categorical_crossentropy'
output_activation = 'softmax'
w_init = 'random_normal' # tf.keras.initializers.HeNormal()
reg_lambda = 5e-4
dropout = 0.5

model_1 = LeNet(conv_layers, pooling_layers, fc_layers,
               input_shape, output_shape, activation, loss,
               output_activation, w_init,reg_lambda, dropout=dropout)

history, end_time = model_1.train(x_train, y_train, x_test, y_test)
test_loss, test_acc = model_1.evaluate(x_test, y_test)
df.loc[len(df)] = [conv_layers, pooling_layers, fc_layers, test_loss, test_acc, end_time]
print(df)
model_1.model.summary()

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.6886 - loss: 1.1455 - val_accuracy: 0.9056 - val_loss: 0.3890
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9153 - loss: 0.3486 - val_accuracy: 0.9312 - val_loss: 0.2950
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9422 - loss: 0.2586 - val_accuracy: 0.9530 - val_loss: 0.2278
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9566 - loss: 0.2127 - val_accuracy: 0.9637 - val_loss: 0.1947
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9653 - loss: 0.1841 - val_accuracy: 0.9662 - val_loss: 0.1793
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9720 - loss: 0.1603 - val_accuracy: 0.9695 - val_loss: 0.1667
Epoch 7/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9743 - loss: 0.1520 - val_accuracy: 0.9729 - val_loss: 0.1542
Epoch 8/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9775 - loss: 0.1376 -

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)                   │ (None, 28, 28, 6)           │             156 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_14                 │ (None, 14, 14, 6)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 14, 14, 16)          │           2,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_15                 │ (None, 7, 7, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 120)                 │          94,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 84)                  │          10,164 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 10)                  │             850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 323,360 (1.23 MB)

 Trainable params: 107,786 (421.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 215,574 (842.09 KB)

### Test: Model 2

In [34]:
conv_layers = [[1,6], [10,6]] # number of filters , 5x5  filers
pooling_layers = [[2, 2],[2,2]] # 2x2 and stride by 2 / we have two pooling layers with 2x2 window  and 2 stride
fc_layers = [10, 10,10] # dense layer
input_shape = (28,28,1)
output_shape = 10
activation = 'relu'
loss = 'categorical_crossentropy'
output_activation = 'softmax'
w_init = 'random_normal' # tf.keras.initializers.HeNormal()
reg_lambda = 5e-4
dropout = 0.5

model_2 = LeNet(conv_layers, pooling_layers, fc_layers,
               input_shape, output_shape, activation, loss,
               output_activation, w_init,reg_lambda, dropout=dropout)

history, end_time = model_2.train(x_train, y_train, x_test, y_test)
test_loss, test_acc = model_2.evaluate(x_test, y_test)
df.loc[len(df)] = [conv_layers, pooling_layers, fc_layers, test_loss, test_acc, end_time]
print(df)
model_2.model.summary()

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.1087 - loss: 2.3043 - val_accuracy: 0.1126 - val_loss: 2.3016
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.1111 - loss: 2.3017 - val_accuracy: 0.1126 - val_loss: 2.3012
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.1131 - loss: 2.3010 - val_accuracy: 0.1126 - val_loss: 2.3012
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1112 - loss: 2.3015 - val_accuracy: 0.1126 - val_loss: 2.3012
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.1115 - loss: 2.3014 - val_accuracy: 0.1126 - val_loss: 2.3012
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.1129 - loss: 2.3012 - val_accuracy: 0.1126 - val_loss: 2.3012
Epoch 7/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1129 - loss: 2.3011 - val_accuracy: 0.1126 - val_loss: 2.3012
Epoch 8/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.1116 - loss: 2.3013

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_17 (Conv2D)                   │ (None, 28, 28, 1)           │              37 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_16                 │ (None, 14, 14, 1)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 14, 14, 10)          │             370 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_17                 │ (None, 7, 7, 10)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 490)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 10)                  │           4,910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 10)                  │             110 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,943 (66.19 KB)

 Trainable params: 5,647 (22.06 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 11,296 (44.13 KB)

### Test: Model 3

In [45]:
conv_layers = [[50,5], [100,5]] # number of filters , 5x5  filers
pooling_layers = [[2, 2],[2,2]] # 2x2 and stride by 2 / we have two pooling layers with 2x2 window  and 2 stride
fc_layers = [200, 100,50] # dense layer
input_shape = (28,28,1)
output_shape = 10
activation = 'relu'
loss = 'categorical_crossentropy'
output_activation = 'softmax'
w_init = 'random_normal' # tf.keras.initializers.HeNormal()
reg_lambda = 5e-4
dropout = 0.5

model_3 = LeNet(conv_layers, pooling_layers, fc_layers,
               input_shape, output_shape, activation, loss,
               output_activation, w_init,reg_lambda, dropout=dropout)

history, end_time = model_3.train(x_train, y_train, x_test, y_test)
test_loss, test_acc = model_3.evaluate(x_test, y_test)
df.loc[len(df)] = [conv_layers, pooling_layers, fc_layers, test_loss, test_acc, end_time]
print(df)
model_3.model.summary()

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.7683 - loss: 1.5443 - val_accuracy: 0.9617 - val_loss: 0.5265
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.9631 - loss: 0.4717 - val_accuracy: 0.9734 - val_loss: 0.3332
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9719 - loss: 0.3114 - val_accuracy: 0.9727 - val_loss: 0.2593
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9765 - loss: 0.2290 - val_accuracy: 0.9743 - val_loss: 0.2050
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9802 - loss: 0.1819 - val_accuracy: 0.9799 - val_loss: 0.1648
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9814 - loss: 0.1563 - val_accuracy: 0.9808 - val_loss: 0.1526
Epoch 7/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9819 - loss: 0.1433 - val_accuracy: 0.9830 - val_loss: 0.1365
Epoch 8/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9834 - loss: 0.13

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)                   │ (None, 28, 28, 50)          │           1,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_22                 │ (None, 14, 14, 50)          │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 14, 14, 100)         │         125,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_23                 │ (None, 7, 7, 100)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 4900)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 200)                 │         980,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 10)                  │             510 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,396,782 (12.96 MB)

 Trainable params: 1,132,260 (4.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,264,522 (8.64 MB)

### Summary of 3 different models

In [49]:
print(df)

            cov_layers    pooling_layers       fc_layers      Loss  Accuracy  \
0    [[6, 5], [16, 5]]  [[2, 2], [2, 2]]       [120, 84]  0.098080  0.984952   
1    [[1, 6], [10, 6]]  [[2, 2], [2, 2]]    [10, 10, 10]  2.301179  0.112571   
2  [[50, 5], [100, 5]]  [[2, 2], [2, 2]]  [200, 100, 50]  0.094274  0.987143   

         Time  
0  418.178465  
1  441.158424  
2  475.297429  


#### Results:

The first model is the one from the class / slides from week 3 module, this is my baseline model to compare my other test.

The second model I made the convolutional layers only 1 filter of 6x6 filter and then 10 filters of 6x6 filter. The same pooling layers and lowered the dense layers to [10, 10, 10]. The results is a greater loss value and a significally lower accuracy of only 11.2% so you need more filters for the model to have a better accuracy and loss value.

The third model I made the convolutional layers with 50 filters of 5x5 filter and then 100 filters of 5x5. The same pooling layers and increased the dense layers to be [200,100, 50]. The results is an loss value and accuracy similar to the first model. So adding a lot more convolutional layers and dense layers does not make the loss value and accuracy significally better.
